# Launch analytics pipeline on tasks without flag

In .env

```
EXTRACTOR_SECRET_KEY=
EXTRACTOR_URL=
MONGODB_URL=
```

Later on, in ORGS_TO_PROCESS, put an org id in it. 

In [ ]:
# Set current working dir
import os
from dotenv import load_dotenv

# Path with .env
os.chdir("/Users/nicolasoulianov/phospho/phospho/backend/scripts")

load_dotenv()

In [ ]:
import pymongo

# Initialize pymongo client
client = pymongo.MongoClient(os.getenv('MONGODB_URL'))
db = client[os.getenv('MONGODB_NAME')]

In [ ]:
db.name

In [ ]:
import datetime 

# Get current date
date = datetime.datetime.now()

# Get the timestamps from 2 weeks ago

two_weeks_ago = date - datetime.timedelta(weeks=2)
two_weeks_ago_timestamp = int(two_weeks_ago.timestamp())

In [ ]:
two_weeks_ago_timestamp

In [ ]:
ORGS_TO_PROCESS = [
    ""
]

In [ ]:
# Fetch all tasks that don't have a flag set or where flag is None
tasks = list(
    db.tasks.find(
        {
            "$and": [
                {"$or": [{"flag": {"$exists": False}}, {"flag": None}]},
                # {"created_at": {"$gte": two_weeks_ago_timestamp}},
                {"org_id": {"$in": ORGS_TO_PROCESS}}
            ]
        }
    )
)

In [ ]:
len(tasks)

In [ ]:
import requests
from tqdm import tqdm

# Call the extractor API on every task

# for task in tqdm(tasks):
#     if "_id" in task:
#         task.pop("_id")
#     response = requests.post(
#         f"{os.getenv('EXTRACTOR_URL')}/v1/pipelines/main/task",
#         json={"task": task},
#         headers={
#             "Content-Type": "application/json",
#             "Authorization": f"Bearer {os.getenv('EXTRACTOR_SECRET_KEY')}",
#         },
#     )

# Rewrite this code to be able to run it in parallel

import concurrent.futures

def process_task(task):
    if "_id" in task:
        task.pop("_id")
    response = requests.post(
        f"{os.getenv('EXTRACTOR_URL')}/v1/pipelines/eval/task",
        json={"task": task, "save_results": True},
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {os.getenv('EXTRACTOR_SECRET_KEY')}",
        },
    )
    return response

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_task, tasks))

results
